## How to attach to TIA and download 

The following code demonstrates how to download to a PLC. 

In this example I used PLCsim:
1. Right click the projectname in the project tree and click properties
2. Select the Protection Tab
3. Check "Support simulation during block compilation
4. Start PLCsim

The code is written for demonstration purposes. If you want to download to a real PLC look at the print outs and change accordingly to match the desired interface

In [1]:
import clr
clr.AddReference('C:\\Program Files\\Siemens\\Automation\\Portal V16\PublicAPI\\V16\\Siemens.Engineering.dll')
from System.IO import DirectoryInfo, FileInfo
import Siemens.Engineering as tia
import Siemens.Engineering.Download as dl



In [2]:
# Connecting to a running instance of TIA with an open project

processes = tia.TiaPortal.GetProcesses() # Making a list of all running processes
print (processes)
process = processes[0]                   # Just taking the first process as an example
mytia = process.Attach()
myproject = mytia.Projects[0]


In [3]:
# Getting the downloadprovider of the first PLC. NOTE change this by iterating over devices in a real project
# Iterating through all available  
download_provider = tia.IEngineeringServiceProvider(myproject.Devices[0].DeviceItems[1]).GetService[dl.DownloadProvider]()
configuration = download_provider.Configuration
for i in configuration.Modes:
    print (i.Name)


PN/IE


In [4]:
# accessing directly first element, and printing to verify Change to other if needed
configuration_mode = configuration.Modes[0] 
print(configuration_mode.Name)


PN/IE


In [5]:
#Iterating to see available
for i in configuration_mode.PcInterfaces:
    print(i.Name)

PLCSIM


In [6]:
#Accessing first elements directly, change if needed.
target_configuration = configuration_mode.PcInterfaces[0].TargetInterfaces[0]
print(target_configuration.Name)

1 X1


In [7]:
def PreConfigureDownload(downloadConfiguration):
   
    if (type(downloadConfiguration) is dl.Configurations.StopModules):
        if (downloadConfiguration != None):
            downloadConfiguration.CurrentSelection = dl.Configurations.StopModulesSelections.StopAll
    
    if (type(downloadConfiguration) is dl.Configurations.AlarmTextLibrariesDownload):
        if (downloadConfiguration != None):
            downloadConfiguration.CurrentSelection = dl.Configurations.AlarmTextLibrariesDownloadSelections.ConsistentDownload

    if (type(downloadConfiguration) is dl.Configurations.BlockBindingPassword):
        if (downloadConfiguration != None):
            password = None #update if applicable
            downloadConfiguration.SetPassword(password)                        

    if (type(downloadConfiguration) is dl.Configurations.CheckBeforeDownload):
        if (downloadConfiguration != None):
            downloadConfiguration.Checked = true

    if (type(downloadConfiguration) is dl.Configurations.ConsistentBlocksDownload):
        if (downloadConfiguration != None):
            downloadConfiguration.CurrentSelection = dl.Configurations.ConsistentBlocksDownloadSelections.ConsistentDownload

    if (type(downloadConfiguration) is dl.Configurations.ModuleWriteAccessPassword):
        if (downloadConfiguration != None):
            password = None;  #update if applicable
            downloadConfiguration.SetPassword(password)

    

In [8]:
def PostConfigureDownload(downloadConfiguration):
    
    if (type(downloadConfiguration) is dl.Configurations.StartModules):
        if (downloadConfiguration != None):
            downloadConfiguration.CurrentSelection = dl.Configurations.StartModulesSelections.StartModule
   

In [9]:

pre = dl.DownloadConfigurationDelegate(PreConfigureDownload)
post = dl.DownloadConfigurationDelegate(PostConfigureDownload)
result = download_provider.Download(target_configuration,pre, post, dl.DownloadOptions.Hardware | dl.DownloadOptions.Software )